<a href="https://colab.research.google.com/github/AINACIOMENDONCA/MYDASHBOARD/blob/master/Portal_Animal_Campinas_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
# 22/09/2022
# Author: Andrea Inácio Mendonça

# WebScraping do Portal Animal da cidade de Campinas
# Utilizando BeautifulSoup e Request
# Integrando num banco de dados SQLite

# Link original:
# Departamento de Proteção e Bem Estar Animal - Prefeitura de Campinas
# https://portalanimal.campinas.sp.gov.br/adocao

# Assunto:
# Animais (principalmente cachorros e gatos) resgatados em situação de rua, machucados,
# atropelados, ou em outras situações. Disponíveis para adoção.

# O objetivo é raspar o Portal Animal e exportar todos os animais disponíveis para
# adoção em um banco de dados. A partir daí consultar características que interessam a
# quem procura um determinado tipo de cachorro ou gato para adoção, de forma mais rápida 
# e assertiva.


In [75]:
import pandas as pd
import sqlite3
import requests as rq
import re
import json
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go

In [76]:
db_doacoes_df = pd.DataFrame (columns=['nome','especie','sexo','idade','porte','extra_info'])
db_doacoes_df.head()

,nome,especie,sexo,idade,porte,extra_info


In [77]:
#registro_animal = []

def registro(nome,especie,sexo,porte,idade,extra_info):
  registro_animal.append (nome)
  registro_animal.append (especie)
  registro_animal.append (sexo)
  registro_animal.append (idade)
  registro_animal.append (porte)
  registro_animal.append (extra_info)

In [78]:
db_doacoes = []

for count in range(0,12):
  url = rq.get (f'https://portalanimal.campinas.sp.gov.br/adocao?field_especie_adocao_target_id=All&field_sexo_adocao_value=All&field_idade_estimada_adocao_target_id=All&field_porte_tamanho_adocao_target_id=All&page={count}')
  soup = BeautifulSoup (url.content, 'html.parser')

  for animal in soup.find_all('div',attrs={"class":"views-field views-field-title my-2 text-center"}):
      # Cria lista para cada animal
      registro_animal = []

      nome = animal.text
      # Captura da proxima classe onde se encontram as demais informacoes sobre o animal
      aux = animal.next.next.next.text
      especie = aux.split(',')[0]  #especie:  cao, gato, cavalo, passaro, etc
      sexo = aux.split(',')[1]   #sexo:  macho, femea
      porte = aux.split(',')[2]  #porte
      idade = aux.split(',')[3]  #idade
      extra_info = aux.split(',')[4]

      #print(nome,especie,sexo,idade,porte,"\n***********\n")
      registro (nome, especie, sexo, porte, idade, extra_info)
      
      # Acrescenta lista registro_animal com 5 atributos na lista principal db_doacoes
      db_doacoes.append(registro_animal)
      
      # Acrescenta lista registro_animal com 5 atributos na lista principal db_doacoes
      #db_doacoes.append(registro_animal (nome, especie, sexo, porte, idade, extra_info)) 


In [79]:
db_doacoes

[['Sujento', ' Gato', ' Macho', ' Adulto', ' Pequeno', ' Agressivo/Feral'],
 ['Nelson', ' Cachorro', ' Macho', ' Adulto', ' Médio', ' Dócil'],
 ['Juninho', ' Cachorro', ' Macho', ' Adulto', ' Médio', ' Brincalhão'],
 ['Mimosa', ' Gato', ' Fêmea', ' Senil ( Idoso )', ' Médio', ' Tranquilo\n'],
 ['Soneca', ' Cachorro', ' Macho', ' Adulto', ' Pequeno', ' Carinhoso'],
 ['Draco', ' Gato', ' Macho', ' Adulto', ' Pequeno', ' Agressivo/Feral'],
 ['Moisés', ' Cachorro', ' Macho', ' Senil ( Idoso )', ' Médio', ' Carinhoso'],
 ['Zeca', ' Cachorro', ' Macho', ' Jovem', ' Médio', ' Carinhoso'],
 ['Aninha', ' Cachorro', ' Fêmea', ' Jovem', ' Pequeno', ' Dócil'],
 ['Maria de Nazaré', ' Cachorro', ' Fêmea', ' Jovem', ' Médio', ' Dócil'],
 ['Madalena', ' Cachorro', ' Fêmea', ' Filhote', ' Médio', ' Brincalhão'],
 ['Marcos', ' Cachorro', ' Macho', ' Filhote', ' Médio', ' Carinhoso'],
 ['Lucas', ' Cachorro', ' Macho', ' Filhote', ' Médio', ' Carinhoso'],
 ['Matheus', ' Cachorro', ' Macho', ' Filhote', ' 

In [80]:
db_doacoes_df = pd.DataFrame (db_doacoes, columns=['nome','especie','sexo','idade','porte','extra_info'])
db_doacoes_df

,nome,especie,sexo,idade,porte,extra_info
0,Sujento,Gato,Macho,Adulto,Pequeno,Agressivo/Feral
1,Nelson,Cachorro,Macho,Adulto,Médio,Dócil
2,Juninho,Cachorro,Macho,Adulto,Médio,Brincalhão
3,Mimosa,Gato,Fêmea,Senil ( Idoso ),Médio,Tranquilo\n
4,Soneca,Cachorro,Macho,Adulto,Pequeno,Carinhoso
...,...,...,...,...,...,...
240,CONAN,Gato,Macho,Jovem,Médio,Assustado\n
241,PIRATA,Gato,Macho,Adulto,Médio,Assustado
242,ASLAN,Gato,Macho,Adulto,Médio,Assustado
243,LUCK,Cachorro,Macho,Adulto,Médio,Carinhoso


In [81]:
## SQL - db_doacoes
## explorar comandos - QUERIES

db_doacoes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   nome        245 non-null    object
 1   especie     245 non-null    object
 2   sexo        245 non-null    object
 3   idade       245 non-null    object
 4   porte       245 non-null    object
 5   extra_info  245 non-null    object
dtypes: object(6)
memory usage: 11.6+ KB


In [82]:
# Conta quantos registros para cada espécie

db_doacoes_df.especie.value_counts()

 Cachorro    125
 Gato        120
Name: especie, dtype: int64

In [83]:
# Grafico pizza mostra que 51% dos animais para adoção são cães.

fig = px.pie (db_doacoes_df, names='especie', title='Animais para Adoção vs Espécie',)
fig.show()

In [84]:
# Conta quantos registros para cada Sexo

db_doacoes_df.sexo.value_counts()

 Macho    172
 Fêmea     73
Name: sexo, dtype: int64

In [85]:
# Grafico pizza mostra que mais de 70,2% dos animais para adoção são machos.

fig = px.pie (db_doacoes_df, names='sexo', title='Animais para Adoção vs Sexo',)
fig.show()

In [86]:
db_doacoes_df.describe(include=['object'])

,nome,especie,sexo,idade,porte,extra_info
count,245,245,245,245,245,245
unique,245,2,2,4,3,15
top,Sujento,Cachorro,Macho,Adulto,Médio,Dócil
freq,1,125,172,141,143,50


In [87]:
# Cria banco de dados Portal_Animal_Campinas

db = sqlite3.connect('portal_animal_campinas')

In [88]:
# Cria tabela doacoes

db_doacoes_df.to_sql('adocoes', db, index=False, if_exists='replace')

In [89]:
# Exporta banco de dados para o formato Planilha do Excel

db_doacoes_df.to_excel('adocoes_portal_animal_campinas.xlsx')

In [90]:
# Cria funcao para chamada do comando SQL

def executaQuery (query):
  return pd.read_sql(query,db)

In [91]:
# Chama funcao e o comando SQL como argumento

executaQuery('select * from adocoes')

,nome,especie,sexo,idade,porte,extra_info
0,Sujento,Gato,Macho,Adulto,Pequeno,Agressivo/Feral
1,Nelson,Cachorro,Macho,Adulto,Médio,Dócil
2,Juninho,Cachorro,Macho,Adulto,Médio,Brincalhão
3,Mimosa,Gato,Fêmea,Senil ( Idoso ),Médio,Tranquilo\n
4,Soneca,Cachorro,Macho,Adulto,Pequeno,Carinhoso
...,...,...,...,...,...,...
240,CONAN,Gato,Macho,Jovem,Médio,Assustado\n
241,PIRATA,Gato,Macho,Adulto,Médio,Assustado
242,ASLAN,Gato,Macho,Adulto,Médio,Assustado
243,LUCK,Cachorro,Macho,Adulto,Médio,Carinhoso


In [92]:
# Consulta nome, idade e extra_info somente de " Cachorro, Femea, Pequeno "
#  atribui variável query

query = '''select nome, idade, extra_info from adocoes 
            where especie LIKE "%Cachorro" 
            AND sexo LIKE "%Fêmea" 
            AND  porte LIKE "%Pequeno" '''
executaQuery(query)

,nome,idade,extra_info
0,Aninha,Jovem,Dócil
1,Loba,Jovem,Assustado\n
2,Flora,Senil ( Idoso ),Carinhoso
3,ISA,Adulto,Obediente


In [93]:
# Consulta as opçoes de temperamento (extra_info) dos animais do banco de dados

executaQuery('select distinct extra_info from adocoes')

,extra_info
0,Agressivo/Feral
1,Dócil
2,Brincalhão
3,Tranquilo\n
4,Carinhoso
5,Assustado\n
6,Dócil\n
7,Arredio
8,Assustado
9,Obediente
